In [1]:
%matplotlib inline
import pandas as pd

## Expand data

In [2]:
compras_df = pd.read_csv('./data/clean_compras.csv')

In [3]:
compras_df.dtypes

NUM BEN           int64
BENEFICIARIO     object
RFC              object
FACTURA          object
CONTRATO         object
CONCEPTO         object
TIPO             object
# OP            float64
FECHA            object
IMPORTE         float64
SHEET            object
dtype: object

In [5]:
compras_df['FECHA'] = pd.to_datetime(compras_df['FECHA'])

In [6]:
compras_df['DAY'] = compras_df['FECHA'].dt.day

In [7]:
compras_df['MONTH'] = compras_df['FECHA'].dt.month

In [8]:
compras_df['YEAR'] = compras_df['FECHA'].dt.year

## Project variable between each other

In [9]:
beneficiario_sales = compras_df['BENEFICIARIO'].value_counts().to_dict()

In [11]:
list(beneficiario_sales.keys())[0:5]

['CAMARA MEXICANA DE LA INDUSTRIA DE LA CONSTRUCCION',
 'AUTOKAM REGIOMONTANA,S.A.DE C.V.',
 'CAR ONE MONTERREY, S.A. DE C.V.',
 'PEREZ RIOS RAMIRO',
 'S.I.M.E.P.R.O.D.E.']

In [13]:
compras_df['BENEFICIARIO_SALES'] = compras_df['BENEFICIARIO'].apply(lambda x: beneficiario_sales[x])

In [14]:
sheet_mean_compras = compras_df.groupby('SHEET')['IMPORTE'].mean()

In [15]:
sheet_mean_compras.to_dict()

{'COMUNICACION': 109705.34366197183,
 'CONTRATISTAS Y FDO FED': 474566.76578729285,
 'GASTOS VARIOS': 881659.7317971016,
 'SERV PPROF': 229592.16666666666,
 'SERV. PERS.': 59623.40521739131}

In [16]:
compras_df['IMPORTE_GASTO_PROMEDIO'] = compras_df['SHEET'].apply(lambda x: sheet_mean_compras[x])

## We bring extra data to enrich using linkedin

In [17]:
pd.DataFrame(compras_df['BENEFICIARIO'].unique()).to_csv('./data/unique_beneficiario.csv')

In [19]:
# Loading
linkedin_mty = pd.read_csv('./data/linkedin_mty.csv')
linkedin_mty.head()

,url,description,title,query,timestamp,error
0,https://www.linkedin.com/in/rosa-lorena-valdez...,Ve el perfil de Rosa Lorena Valdez Miranda en ...,Rosa Lorena Valdez Miranda - Analista de Deuda...,Rosa Lorena Valdez Miranda,2021-03-08T13:06:25.587Z,NaN
1,https://www.linkedin.com/in/roque-ya%C3%B1ez-r...,"Ve el perfil de Roque Yañez Ramos en LinkedIn,...",Roque Yañez Ramos - Síndico Primero - Gobierno...,Roque Yañez Ramos,2021-03-08T13:06:52.599Z,NaN
2,https://www.linkedin.com/in/perla-cordova-vill...,Ve el perfil de Perla Cordova Villegas en Link...,Perla Cordova Villegas - Profesor de Tiempo Co...,Laura Perla Cordova Rodriguez,2021-03-08T13:07:18.940Z,NaN
3,https://www.linkedin.com/in/lauren-coronado-b3...,"View Lauren Coronado’s profile on LinkedIn, th...",Lauren Coronado - Substitute Teacher - IRVING ...,Rosa Ofelia Coronado Flores,2021-03-08T13:07:45.390Z,NaN
4,https://www.linkedin.com/in/alvaro-palomo,Ve el perfil de Alvaro Palomo Osuna en LinkedI...,Alvaro Palomo Osuna - Jefe de proyecto .NET - ...,Alvaro Flores Palomo,2021-03-08T13:08:11.744Z,NaN


In [20]:
has_site = {}
for i, ele in linkedin_mty.fillna(True).iterrows():
    has_site[ele['query']] = True if ele['error'] == True else False

In [22]:
compras_df['HAS_LINKEDIN_SITE'] = compras_df['BENEFICIARIO'].apply(lambda x: has_site[x.replace('  ', ' ')])

In [24]:
compras_df['HAS_LINKEDIN_SITE'].value_counts()

HAS_LINKEDIN_SITE
True     1392
False     148
Name: count, dtype: int64

In [25]:
compras_df.to_csv('./data/compras_df_enriched.csv', index=None)